# Import library

In [27]:
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient 

## Connect web

In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36", 
           "Accept-Encoding":"gzip, deflate, br, zstd", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
           "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(url, headers=headers )

soup = BeautifulSoup(page.content, 'html.parser')

In [30]:
# print(soup)

In [35]:
Company=soup.find_all('table')[0]

## Connect MongoDB

In [48]:
client = MongoClient('localhost',27017)
db=client.largest_company 

In [36]:
column_data = Company.find_all('tr')

## Find table name 

In [38]:
world_title=Company.find_all('th')
world_table_titles = [title.text.strip() for title in world_title]

print(world_table_titles)

['Rank', 'Name', 'Industry', 'Revenue (USD millions)', 'Revenue growth', 'Employees', 'Headquarters']


In [47]:
db.company.drop()

## Loop through each row of data, and put it into mongodb

In [49]:
for row in column_data[1:]:
    cols = [td.text.strip() for td in row.find_all('td')]

    
    while len(cols) < len(world_table_titles):
        cols.append("")

    document = dict(zip(world_table_titles, cols))
    db.company.insert_one(document)  